In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string
import matplotlib.pylab as plt
from numpy import linalg as LA
%matplotlib
import seaborn as sns
import os

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Using matplotlib backend: MacOSX
sample_submission.csv
stanfordData_train1.nlp
test.csv
test_collins_duffy
test_dean
train.csv
train_collins_duffy
train_dean



## Extract, Transform, Load

In [2]:
from sklearn.model_selection import train_test_split

def read_data(path_to_file):
    df = pd.read_csv(path_to_file)
    print ("Shape of base training File = ", df.shape)
    print("Shape of base training data after cleaning = ", df.shape)
    return df

df_train = read_data("input/train.csv")
df_submit = read_data("input/test.csv")

# Print the column names
print (df_train.columns.values.tolist())

Shape of base training File =  (404290, 6)
Shape of base training data after cleaning =  (404290, 6)
Shape of base training File =  (2345796, 3)
Shape of base training data after cleaning =  (2345796, 3)
['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate']


## Load the Collins Duffy Training Features 

In [3]:
df_features = None
flip = True
base = 'input/train_collins_duffy/'
count = 0
for filename in os.listdir(base):
    if filename.endswith(".csv"): 
        tmpFrame = pd.read_csv(os.path.join(os.getcwd(), base, filename))
        print(os.path.join(os.getcwd(), base, filename))
        tmpFrame = tmpFrame.rename(columns={'cdNorm_st': filename.replace(".csv","")})
        if flip:
            df_features = tmpFrame
            flip = False
        else:
            df_features = df_features.merge(tmpFrame,how='inner',on='id')
        count+=1
        continue
    else:
        continue

/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_1e-1_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_1e-1_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_1e-2_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_1e-2_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_1e0_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_1e0_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_2e-1_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_2e-1_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_5e-1_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_5e-1_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/train_collins_duffy/sd_5e-2_sst.csv
/Users/deanfulgoni/Documents

## Load the Collins Duffy Test Features

In [4]:
df_features_test = None
flip = True
base = 'input/test_collins_duffy/'
count = 0
for filename in os.listdir(base):
    if filename.endswith(".csv"): 
        tmpFrame = pd.read_csv(os.path.join(os.getcwd(), base, filename))
        print(os.path.join(os.getcwd(), base, filename))
        tmpFrame = tmpFrame.rename(columns={'cdNorm_st': filename.replace(".csv","")})
        if flip:
            df_features_test = tmpFrame
            flip = False
        else:
            df_features_test = df_features_test.merge(tmpFrame,how='inner',on='id')
        count+=1
        continue
    else:
        continue

/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_1e-1_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_1e-1_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_1e-2_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_1e-2_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_1e0_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_1e0_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_2e-1_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_2e-1_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_5e-1_sst.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_5e-1_st.csv
/Users/deanfulgoni/Documents/Code/quora_kaggle/input/test_collins_duffy/sd_5e-2_sst.csv
/Users/deanfulgoni/Documents/Code/quora

## Load in Dean's Semantic Features

In [5]:
df_semantic = pd.read_csv('input/train_dean/dean_train_features.csv')
df_semantic.drop('is_duplicate',inplace=True,axis=1)

## Training the Model

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, classification_report, auc, roc_curve, log_loss, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2

# Read in the pre-built features for the training set
df_train = df_train.merge(df_features,how='inner',on='id')
df_train = df_train.merge(df_semantic,how='inner',on='id')

# Methodology - fill with zeroes Any row that crashed the Stanford CoreNLP parser must have been empty string
df_train.fillna(value=0, inplace=True)

In [7]:
df_train.columns

Index(['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate',
       'sd_1e-1_sst', 'sd_1e-1_st', 'sd_1e-2_sst', 'sd_1e-2_st', 'sd_1e0_sst',
       'sd_1e0_st', 'sd_2e-1_sst', 'sd_2e-1_st', 'sd_5e-1_sst', 'sd_5e-1_st',
       'sd_5e-2_sst', 'sd_5e-2_st', 'sd_8e-1_sst', 'sd_8e-1_st', 'S_sem_sim',
       'T_sem_sim', 'sem_sim', 'S_unmatch_n', 'T_unmatch_n', 'S_unmatch_n_p',
       'T_unmatch_n_p', 'S_unmatch_a', 'T_unmatch_a', 'S_unmatch_a_p',
       'T_unmatch_a_p', 'S_unmatch_v', 'T_unmatch_v', 'S_unmatch_v_p',
       'T_unmatch_v_p', 'S_unmatch_pp', 'T_unmatch_pp', 'S_unmatch_pp_p',
       'T_unmatch_pp_p', 'S_unmatch_wp', 'T_unmatch_wp', 'S_unmatch_wp_p',
       'T_unmatch_wp_p', 'S_unmatch_num', 'T_unmatch_num', 'S_unmatch_num_p',
       'T_unmatch_num_p', 'S_unmatch_ner', 'T_unmatch_ner', 'len_dif',
       'len_dif_p'],
      dtype='object')

## Adding More Features

In [ ]:
# # Question Lengths
# df_train['q1len'] = df_train['question1'].str.len()
# df_train['q2len'] = df_train['question2'].str.len()

# df_submit['q1len'] = df_submit['question1'].str.len()
# df_submit['q2len'] = df_submit['question2'].str.len()

# # Normalized Word Share
# def number_of_words(row):
#     try:
#         nwords = len(row.split(" "))
#         return nwords
#     except:
#         return 0

# # Normalized Word Share
# def normalized_word_share(row):
#     try:
#         w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
#         w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
#         return 1.0 * len(w1 & w2)/(len(w1) + len(w2))
#     except:
#         return 0

# def word_diff(row):
#     try:
#         diff = abs(row['q1_n_words']-row['q2_n_words'])
#         return diff
#     except:
#         return 0
    
# # Number of Words
# df_train['q1_n_words'] = df_train['question1'].apply(number_of_words)
# df_train['q2_n_words'] = df_train['question2'].apply(number_of_words)

# df_submit['q1_n_words'] = df_submit['question1'].apply(number_of_words)
# df_submit['q2_n_words'] = df_submit['question2'].apply(number_of_words)
    
# # Normalized Word share
# df_train['word_share'] = df_train.apply(normalized_word_share, axis=1)
# df_submit['word_share'] = df_submit.apply(normalized_word_share, axis=1)

# df_train['word_diff'] = df_train.apply(word_diff, axis=1)
# df_submit['word_diff'] = df_submit.apply(word_diff, axis=1)

## Extract the Features

In [8]:
select_cols = list(df_train.columns[6:])
df_y = df_train['is_duplicate']
df_X = df_train[select_cols].copy()

df_X.fillna(value=0, inplace=True)

In [9]:
X = df_X.values
y = df_y.values

X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.10, random_state=24)

X_train_mat = X_train.values
y_train_mat = y_train.values

X_train_mat = MinMaxScaler().fit_transform(X_train_mat)

X_train.shape, X_test.shape, y_train.shape, y_test.shape
X.shape, y.shape

((404282, 45), (404282,))

## Feature Selection

In [ ]:
selector = SelectKBest(chi2, k=3)
X_selected = selector.fit_transform(X, y)
column_indexer = selector.get_support()
X_df_selected = df_X.loc[:,column_indexer]
X_df_selected.columns

## Feature Plotting

In [ ]:
plt.figure(figsize=(15, 5))
plt.hist(df_X[df_train['is_duplicate'] == 0]['sem_sim'], bins=20, normed=False, label='Not Duplicate')
plt.hist(df_X[df_train['is_duplicate'] == 1]['sem_sim'], bins=20, normed=False, alpha=0.6, label='Duplicate')
plt.legend()
plt.title('Label Distribution Over Semantic Similarity of Aligned Words', fontsize=15)
plt.xlabel('Normalized Percentage of Aligned Pairs', fontsize=15)

In [ ]:
plt.figure(figsize=(15, 5))
plt.hist(df_X[df_train['is_duplicate'] == 0]['sd_1e-1_sst'], bins=20, normed=False, label='Not Duplicate')
plt.hist(df_X[df_train['is_duplicate'] == 1]['sd_1e-1_sst'], bins=20, normed=False, alpha=0.6, label='Duplicate')
plt.legend()
plt.title('Label Distribution Over SST Tree Kernel, Lambda = 0.1', fontsize=15)
plt.xlabel('Normalized Score', fontsize=15)

In [ ]:
plt.figure(figsize=(15, 5))
plt.hist(df_X[df_train['is_duplicate'] == 0]['S_unmatch_ner'], bins=20, normed=False, label='Not Duplicate')
plt.hist(df_X[df_train['is_duplicate'] == 1]['S_unmatch_ner'], bins=20, normed=False, alpha=0.6, label='Duplicate')
plt.legend()
plt.title('Label Distribution Over Number Unmatched Named Entities', fontsize=15)
plt.xlabel('Number Unmatched Named Entities (cutoff=3)', fontsize=15)

## Logistic Regression Training

In [ ]:
#grid = {
 #   'C': [1e-3, 1e0, 1e1],
#    'penalty': ['l1', 'l2']
#}
#logreg_cv = GridSearchCV(logreg_clf, grid, scoring='neg_log_loss', n_jobs=-1, verbose=1)

# Print the best cross-validation parameters
#print(logreg_cv.best_params_)
#print(logreg_cv.best_estimator_.coef_)

In [ ]:
# Best was found to be l2 penalty (default) and C=10
logreg_clf = LogisticRegression(C=10)
y_pred_logreg = cross_val_predict(logreg_clf, X, y)
log_loss_scores = cross_val_score(logreg_clf, X, y, scoring='neg_log_loss')
ll = log_loss_scores.mean()

In [ ]:
acc = accuracy_score(y, y_pred_logreg)
prec = precision_score(y, y_pred_logreg)
recall = recall_score(y, y_pred_logreg)
f1 = f1_score(y, y_pred_logreg)
print(acc, prec, recall, f1)
print(ll)

## Random Forest Training

In [ ]:
rf_clf = RandomForestClassifier()
grid = {
    'n_estimators': [200],
    'max_features': [5, 10, 20]
}
rf_cv = GridSearchCV(rf_clf, grid, scoring='neg_log_loss', n_jobs=-1, verbose=1)
rf_cv.fit(X_train_mat, y_train_mat)
print(rf_cv.best_params_)

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=200, max_features=5)
y_pred_rf = cross_val_predict(rf_clf, X, y)
log_loss_scores = cross_val_score(rf_clf, X, y, scoring='neg_log_loss')
ll = log_loss_scores.mean()

In [ ]:
acc = accuracy_score(y, y_pred_rf)
prec = precision_score(y, y_pred_rf)
recall = recall_score(y, y_pred_rf)
f1 = f1_score(y, y_pred_rf)
print(acc, prec, recall, f1)
print(ll)

## ROC

In [ ]:
colors = ['r', 'g', 'b', 'y', 'k', 'c', 'm', 'brown', 'r']
lw = 1
Cs = [1e-6, 1e-3, 1e0, 1e1]

plt.figure(figsize=(12,8))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for different classifiers')

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')

labels = []
for idx, C in enumerate(Cs):
    clf = LogisticRegression(C = C)
    clf.fit(X_train_mat, y_train_mat)
    print("C: {}, parameters {} and intercept {}".format(C, clf.coef_, clf.intercept_))
    fpr, tpr, _ = roc_curve(y_test, clf.predict_proba(X_test)[:,1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=colors[idx])
    labels.append("C: {}, AUC = {}".format(C, np.round(roc_auc, 4)))

plt.legend(['random AUC = 0.5'] + labels)

## Precision-Recall

In [ ]:
pr, re, _ = precision_recall_curve(y_test, cv.best_estimator_.predict_proba(X_test)[:,1])
plt.figure(figsize=(12,8))
plt.plot(re, pr)
plt.title('PR Curve (AUC {})'.format(auc(re, pr)))
plt.xlabel('Recall')
plt.ylabel('Precision')

## Load in test set features

In [10]:
df_semantic_test = pd.read_csv('input/test_dean/dean_test_features.csv')

In [11]:
# Read in the pre-built features for the test set
df_submit = df_submit.merge(df_features_test,how='left',left_on='test_id',right_on='id')
# df_train = df_train.merge(df_semantic,how='inner',on='id')
df_submit.drop(['id'],inplace=True,axis=1)
df_submit = df_submit.merge(df_semantic_test,how='left',left_on='test_id',right_on='id')
df_submit.drop(['id'],inplace=True,axis=1)

# Methodology - fill with zeroes Any row that crashed the Stanford CoreNLP parser must have been empty string
df_submit.fillna(value=0, inplace=True)

X_test_set = df_submit[df_submit.columns[3:]].values

## Submissions for Logistic Regression and Random Forest

In [ ]:
logreg_clf = LogisticRegression(C=10)
logreg_clf.fit(X, y)
probs = logreg_clf.predict_proba(X_test_set)
df_submit['is_duplicate'] = probs[:,1]
df_final_submit = df_submit[['test_id','is_duplicate']]
df_final_submit.to_csv("submissions/lr_final.csv",index=False)

In [12]:
rf_clf = RandomForestClassifier(n_estimators=200, max_features=5)
rf_clf.fit(X, y)
probs = rf_clf.predict_proba(X_test_set)
df_submit['is_duplicate'] = probs[:,1]
df_final_submit = df_submit[['test_id','is_duplicate']]
df_final_submit.to_csv("submissions/rf_final.csv",index=False)